Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
import netgen.gui
from ngsolve import *
%gui tk

We generate a 3D geometry and mesh using the constructive solid geometry (CSG) modeler of Netgen:

In [17]:
from netgen.csg import *
geo = CSGeometry()
cube = OrthoBrick (Pnt(0,0,0), Pnt(1,1,1)).bc("outer")
cyl = Cylinder( Pnt(1,0.5,0.5), Pnt(0,0.5,0.5), 0.2).bc("inner")
geo.Add (cube-cyl, col=(0,0,1))
geo.Draw()

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
mesh.Refine()
mesh.Refine()
mesh.Curve(3)
Draw (mesh)

In [18]:
fes = H1(mesh, order=3, dirichlet="outer", flags = { "wb_withoutedges" : True })
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))

f = LinearForm(fes)
f += SymbolicLFI(v)

# c = Preconditioner(a, "local")
# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 1560464 unknowns


assemble system and setup preconditioner in parallel:

In [19]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [20]:
from ngsolve.solvers import CG

with TaskManager():
    CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
        printrates=True, maxsteps=200)
Draw(gfu)

it =  0  err =  0.28712089056269857
it =  1  err =  0.10749286926619048
it =  2  err =  0.08752453658398944
it =  3  err =  0.07039491341804943
it =  4  err =  0.050775993651145934
it =  5  err =  0.03360351715919446
it =  6  err =  0.021700359704191954
it =  7  err =  0.01391397256844124
it =  8  err =  0.009417669959426115
it =  9  err =  0.006567387116831614
it =  10  err =  0.004427831463017223
it =  11  err =  0.00291784182593921
it =  12  err =  0.0019173542581778205
it =  13  err =  0.0012463460302125549
it =  14  err =  0.0008156179258766549
it =  15  err =  0.0005473087950712046
it =  16  err =  0.0003759535430883341
it =  17  err =  0.0002500566767943354
it =  18  err =  0.00016339110772208647
it =  19  err =  0.00010648009727645814
it =  20  err =  6.982404633160817e-05
it =  21  err =  4.651461183373725e-05
it =  22  err =  3.1141188538556944e-05
it =  23  err =  2.0824373861934208e-05
it =  24  err =  1.3745449757482554e-05
it =  25  err =  8.994920227547768e-06
it =  26  